In [ ]:
import DQN
import tqdm as tq
from Environment import *

env = environment()
agent = DQN.Agent(25, 8)

# train part
for epi in tq.tqdm(range(100000)) :
    
    now_state = env.reset()
    done = False
    
    while not done :
        
        action = agent.get_action(now_state, epi)

        next_state, reward, done = env.step(action)
        
        agent.memorize(now_state, action, next_state, reward)
        
        agent.update_q_function()
        
        if done : 
            break
        
        now_state = next_state

In [ ]:
# test part
import os
import torch
import numpy as np
import sklearn.metrics as metrics

os.chdir('C:\\Users\\seo\\Desktop\\Lab\\Projects\\정부) 재활로봇\\임상 data\\lightgbm')
test_data = np.loadtxt('test_1.csv', delimiter=',', dtype=np.float32)
np.random.shuffle(test_data)

test_state = test_data[:, :-1]
test_label = test_data[:, -1].flatten()
estimated_label = []

for idx in tq.tqdm(range(len(test_data))) :
    state = torch.unsqueeze(torch.from_numpy(test_state[idx]).type(torch.FloatTensor).cuda(), 0)
    action = agent.test(state, 1)
    estimated_label.append(action.cpu().numpy()[0])

print('Accuracy Rate')
print(metrics.accuracy_score(test_label, estimated_label), end='\n\n')

print('Confusion Matrix')
print(metrics.confusion_matrix(test_label, estimated_label), end='\n\n')

print('Classification Report')
target_names = [str(i)+' minor' if i in [1,2,3] else str(i)+' Major' for i in range(8)]
print(metrics.classification_report(test_label, estimated_label, target_names=target_names))